# 데이터 병합 및 샘플링 실행 결과

**설명 파일 :** [02_데이터병합및샘플링.md](02_데이터병합및샘플링.md)

## Step 1. 사전 설정

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import gc
import os
import math
import warnings
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

# 파일 위치 사전 정의
warnings.filterwarnings("ignore")
DATA_RAW_DIR = './data/raw'
DATA_OUTPUT_DIR = './data/processed'
ANALYSIS_OUTPUT_DIR = './outputs/tables'
SAMPLING_OUTPUT_DIR = './data/external'

# 한글 폰트 설정
fe = fm.FontEntry(
fname=r'/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf',
name='NanumGothic')
fm.fontManager.ttflist.insert(0, fe)
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'})

# 마이너스 표시 문제
mpl.rcParams['axes.unicode_minus'] = False

print("모듈 import 완료")

모듈 import 완료


## Step 2. 월별 데이터로 병합

In [2]:
## 1. 경기도 카드 소비 데이터 로드

# 작업 해야 하는 파일 불러오기
all_files = os.listdir(DATA_RAW_DIR)
csv_files = sorted([f for f in all_files if f.startswith('tbsh_gyeonggi_day_') and f.endswith('.csv')])

# 이미 작업한 파일 찾기
exist_files = sorted([f for f in os.listdir(DATA_OUTPUT_DIR) if f.startswith('gyeonggi_') and f.endswith('.csv')])
# 이미 작업한 개월 
aleady_processed_months = {f.replace('gyeonggi_', '').replace('.csv', '') for f in exist_files}
# 작업 해야하는 개월
all_months = [x for x in [f"{year}{month:02d}" for year in range(2022, 2026) for month in range(1, 13)] if x <= "202509"]
target_months = sorted([x for x in all_months if x not in aleady_processed_months])

print("경기도 카드 소비 데이터 로드")
print(f"\nLoading {len(csv_files)} CSV files")

## 2. 월별로 파일 그룹화

# 파일명에서 연월 추출하는 함수
def extract_month(filename):
    parts = filename.split('_')
    if len(parts) >= 4:
        return parts[3]
    return None

monthly_files = {}

for filename in csv_files:
    months = extract_month(filename)
    if months:
        monthly_files.setdefault(months, []).append(filename)

print(f"그룹화 완료: {len(monthly_files)}개월")

# 디버깅용 변수
success_count = 0
failed_months = []

경기도 카드 소비 데이터 로드

Loading 459 CSV files
그룹화 완료: 45개월


In [10]:
## 3. 월별 카드 소비 데이터 병합

for month in sorted(target_months):
    month_files = monthly_files[month]
    print(f"\n[{month}] 처리 중... ({len(month_files)}개 파일)")    
    dfs = []
    
    # 월별 데이터 병합 시도 (인코딩 관련 에러 해결 코드 추가)
    for filename in month_files:
        filepath = os.path.join(DATA_RAW_DIR, filename)
        df_temp = None
        for encoding in ['utf-8', 'euc-kr', 'cp949']:
            try:
                df_temp = pd.read_csv(filepath, encoding=encoding)
                print(f"  O {filename:50s} | {len(df_temp):7,d} rows")
                break
            except:
                continue
        if df_temp is not None:
            dfs.append(df_temp)
    
    if len(dfs) > 0:
        df_month = pd.concat(dfs, ignore_index=True)
        
        # 저장 경로
        output_file = os.path.join(DATA_OUTPUT_DIR, f"gyeonggi_{month}.csv")
        
        # csv로 저장
        df_month.to_csv(output_file, index=False)
        
        file_size_mb = os.path.getsize(output_file) / 1024**2
        print(f"  저장 완료: {output_file}")
        print(f"    - 행: {len(df_month):,}")
        print(f"    - 크기: {file_size_mb:.2f} MB")
        
        success_count += 1
        
        # 메모리 정리
        del df_month, dfs
        gc.collect()
    
    else:
        print(f"  X {month}월 데이터 로드 실패")
        failed_months.append(month)

## 4. 완료
print(f"완료: {success_count}개월 저장 완료")

if failed_months:
    print(f"X 실패: {len(failed_months)}개월 ({', '.join(failed_months)})")



[202201] 처리 중... (10개 파일)
  O tbsh_gyeonggi_day_202201_광명시.csv                   | 632,806 rows
  O tbsh_gyeonggi_day_202201_부천시.csv                   | 1,475,049 rows
  O tbsh_gyeonggi_day_202201_수원시.csv                   | 2,116,088 rows
  O tbsh_gyeonggi_day_202201_시흥시.csv                   | 866,931 rows
  O tbsh_gyeonggi_day_202201_안산시.csv                   | 1,149,994 rows
  O tbsh_gyeonggi_day_202201_안양시.csv                   | 1,209,953 rows
  O tbsh_gyeonggi_day_202201_용인시.csv                   | 1,831,905 rows
  O tbsh_gyeonggi_day_202201_포천시.csv                   | 350,242 rows
  O tbsh_gyeonggi_day_202201_하남시.csv                   | 577,059 rows
  O tbsh_gyeonggi_day_202201_화성시.csv                   | 1,269,815 rows
  저장 완료: ./data/processed/gyeonggi_202201.csv
    - 행: 11,479,842
    - 크기: 782.95 MB

[202202] 처리 중... (10개 파일)
  O tbsh_gyeonggi_day_202202_광명시.csv                   | 560,953 rows
  O tbsh_gyeonggi_day_202202_부천시.csv                   | 1,299,014 rows
  O tb

## Step 3. 0값, 음수, 결측치 찾기

In [5]:
# 매출 데이터 등에서 음수인 값을 찾는 함수
def df_negative(df):
    negatives = {}
    for col in df.select_dtypes(include=[np.number]).columns:
        # 0 이하인 값이 있다면 딕셔너리 값 반환
        count = (df[col]<0).sum()
        if count > 0:
            negatives[col] = {
                'count': count,
                'percentage': (count/len(df))*100,
            }

    return negatives

# 0 값이 있는지 찾는 함수
def df_zero(df, columns=None):
    zeros = {}
    # 컬럼 리스트 columns 인수가 None이라면 모든 수치형 컬럼으로 지정
    if columns is None:
        columns = df.select_dtypes(include=[np.number]).columns
    for col in columns:
        count = (df[col] == 0).sum()
        if count > 0:
            zeros[col] = {
                'count' : count,
                'percentage' : (count/len(df))*100
            }
    
    return zeros

In [12]:
## 4. 병합 파일에서 결측치 찾기

processed_files = [f for f in os.listdir(DATA_OUTPUT_DIR) if f.startswith('gyeonggi_') and f.endswith('.csv')]
processed_files = sorted(processed_files)

print("결측치 찾기 시작")
print(f"파일: {len(processed_files)}개")

search_results = []

for i, filename in enumerate(processed_files, 1):
    month = filename.replace('gyeonggi_', '').replace('.csv', '')
    
    try:
        filepath = os.path.join(DATA_OUTPUT_DIR, filename)

        print(f"\n[{i:2d}/{len(processed_files)}] {month} 찾는 중...")
        
        # 파일 읽기
        df = pd.read_csv(filepath)
        
        # 실행
        missing_count = df.isnull().sum().sum()
        missing_by_col = {col: {'count': df[col].isnull().sum(), 
                                'percentage': round((df[col].isnull().sum() / len(df)) * 100, 4)}
                            for col in df.columns if df[col].isnull().sum() > 0}
        
        duplicate_count = df.duplicated().sum()
        negatives = df_negative(df)
        zeros = df_zero(df)
        
        # 결과 저장
        search_report = {
            'month': month,
            'data_shape': {'rows': len(df), 'columns': len(df.columns)},
            'results': {
                'missing_values': {'total_missing': missing_count, 'by_column': missing_by_col},
                'duplicates': {'total_duplicates': duplicate_count, 
                              'percentage': round((duplicate_count / len(df)) * 100, 4)},
                'negative_values': negatives,
                'zero_values': zeros
            }
        }
        
        search_results.append(search_report)
        
        del df
        gc.collect()
    
    except Exception as e:
        print(f"\n X {month} 오류: {str(e)[:40]}")

print("\n O 모든 월 찾기 완료")


결측치 찾기 시작
파일: 45개

[ 1/45] 202201 찾는 중...

[ 2/45] 202202 찾는 중...

[ 3/45] 202203 찾는 중...

[ 4/45] 202204 찾는 중...

[ 5/45] 202205 찾는 중...

[ 6/45] 202206 찾는 중...

[ 7/45] 202207 찾는 중...

[ 8/45] 202208 찾는 중...

[ 9/45] 202209 찾는 중...

[10/45] 202210 찾는 중...

[11/45] 202211 찾는 중...

[12/45] 202212 찾는 중...

[13/45] 202301 찾는 중...

[14/45] 202302 찾는 중...

[15/45] 202303 찾는 중...

[16/45] 202304 찾는 중...

[17/45] 202305 찾는 중...

[18/45] 202306 찾는 중...

[19/45] 202307 찾는 중...

[20/45] 202308 찾는 중...

[21/45] 202309 찾는 중...

[22/45] 202310 찾는 중...

[23/45] 202311 찾는 중...

[24/45] 202312 찾는 중...

[25/45] 202401 찾는 중...

[26/45] 202402 찾는 중...

[27/45] 202403 찾는 중...

[28/45] 202404 찾는 중...

[29/45] 202405 찾는 중...

[30/45] 202406 찾는 중...

[31/45] 202407 찾는 중...

[32/45] 202408 찾는 중...

[33/45] 202409 찾는 중...

[34/45] 202410 찾는 중...

[35/45] 202411 찾는 중...

[36/45] 202412 찾는 중...

[37/45] 202501 찾는 중...

[38/45] 202502 찾는 중...

[39/45] 202503 찾는 중...

[40/45] 202504 찾는 중...

[41/45] 202505 찾는 중..

In [13]:
# 결과 csv 파일로 저장
summary_data = []
for result in search_results:
    missing = result['results']['missing_values']
    duplicate = result['results']['duplicates']
    negative = result['results']['negative_values']
    zero = result['results']['zero_values']

    summary_data.append({
        'month': result['month'],
        'total_rows': result['data_shape']['rows'],
        'missing_count': missing['total_missing'],
        'duplicate_count': duplicate['total_duplicates'],
        'negative_count': sum(v['count'] for v in negative.values()) if negative else 0,
        'zero_count': sum(v['count'] for v in zero.values()) if zero else 0
    })

summary_df = pd.DataFrame(summary_data)
csv_output = os.path.join(ANALYSIS_OUTPUT_DIR, 'searching_summary.csv')
summary_df.to_csv(csv_output, index=False, encoding='utf-8-sig')

print("\n 결과 저장 완료")



 결과 저장 완료


In [6]:
# 결과 보기
summary_df = pd.read_csv(os.path.join(ANALYSIS_OUTPUT_DIR, 'searching_summary.csv'))
summary_df.head()

,month,total_rows,missing_count,duplicate_count,negative_count,zero_count
0,202201,11479842,0,0,0,89
1,202202,10191338,0,0,0,76
2,202203,12106986,0,0,0,115
3,202204,12500585,0,0,0,148
4,202205,13218383,0,0,0,153


In [15]:
# 0 값 제거

drop_target_month = summary_df[summary_df['zero_count'] > 0]['month'].tolist()
print(f'0값 제거 시작')
print(f'0값 제거 대상 월: {drop_target_month}')

# 디버깅용 변수
processed_count = 0

for i, month in enumerate(drop_target_month, 1):
    file_path = os.path.join(DATA_OUTPUT_DIR, f'gyeonggi_{month}.csv')
    print(f"[{i}/{len(drop_target_month)}] {month}...", end=" ")
    
    try:
        df = pd.read_csv(file_path)
        rows_before = len(df)
        
        # 0값 찾아서 제거 (수치형 컬럼만)
        for col in df.select_dtypes(include=[np.number]).columns:
            df = df[df[col] != 0]
            
        rows_after = len(df)
        removed_rows = rows_before - rows_after

        # 원본 파일에 덮어쓰기
        df.to_csv(file_path, index=False, encoding='utf-8-sig')
            
        print(f"완료 ({removed_rows:,}행 제거)")
        processed_count += 1
            
        del df
        gc.collect()

    except Exception as e:
        print(f"X 오류: {str(e)[:40]}")

print(f"처리 완료: {processed_count}개월")


0값 제거 시작
0값 제거 대상 월: [202201, 202202, 202203, 202204, 202205, 202206, 202207, 202208, 202209, 202210, 202211, 202212, 202301, 202302, 202303, 202304, 202305, 202306, 202307, 202308, 202309, 202310, 202311, 202312, 202401, 202402, 202403, 202404, 202410]
[1/29] 202201... 완료 (89행 제거)
[2/29] 202202... 완료 (76행 제거)
[3/29] 202203... 완료 (115행 제거)
[4/29] 202204... 완료 (148행 제거)
[5/29] 202205... 완료 (153행 제거)
[6/29] 202206... 완료 (162행 제거)
[7/29] 202207... 완료 (127행 제거)
[8/29] 202208... 완료 (156행 제거)
[9/29] 202209... 완료 (169행 제거)
[10/29] 202210... 완료 (175행 제거)
[11/29] 202211... 완료 (203행 제거)
[12/29] 202212... 완료 (157행 제거)
[13/29] 202301... 완료 (150행 제거)
[14/29] 202302... 완료 (139행 제거)
[15/29] 202303... 완료 (209행 제거)
[16/29] 202304... 완료 (205행 제거)
[17/29] 202305... 완료 (208행 제거)
[18/29] 202306... 완료 (169행 제거)
[19/29] 202307... 완료 (179행 제거)
[20/29] 202308... 완료 (185행 제거)
[21/29] 202309... 완료 (229행 제거)
[22/29] 202310... 완료 (240행 제거)
[23/29] 202311... 완료 (288행 제거)
[24/29] 202312... 완료 (241행 제거)
[25/29] 20240

## Step 4. 데이터 샘플링

In [16]:
# 최소 표본 크기 계산 후 샘플링 비율 산출

population_size = math.ceil(summary_df['total_rows'].sum()/len(summary_df)) # 모집단의 크기 (월별 데이터의 평균값)
margin_error = 0.05 # 5% 오차 한계
confidence_level = 0.95 # 95% 신뢰도

# 1. 신뢰도를 기반으로 z score를 정한다.
z_score = stats.norm.ppf((1 - confidence_level) / 2) # 신뢰도 95%일 때 1.96

# 2. p값을 0.5로 잡는다. 일반적으로 p값이 0.5일 때 표본이 최대 분산을 갖는다.
p = 0.5

# 3. 일반적으로 n = (z ** 2) * p * (1 - p) / e ** 2 과 같은 방법으로 구한다.
# 무한 모집단 가정 시 표본 크기
n = (z_score**2 * p * (1-p)) / (margin_error**2)

# 4. 유한모집단 수정 계수를 적용.

min_sample_size = math.ceil(n / (1 + ((n - 1) / population_size)))

# 5. 권장 샘플링 비율을 산출한다.
sampling_ratio = (n / population_size) * 100


# 결과 출력
print(f"  모집단의 크기 평균: {population_size}")
print(f"  최소 표본 크기: {min_sample_size}")
print(f"  최소 샘플링 비율: {sampling_ratio:.4f}%")

  모집단의 크기 평균: 12835278
  최소 표본 크기: 385
  최소 샘플링 비율: 0.0030%


In [ ]:
# 계층화 샘플링 함수 정의

from sklearn.model_selection import train_test_split

# 샘플링 할 데이터
dtypes = {
    'card_tpbuz_cd': 'category',# 카드사 업종
    'admi_cty_no': 'int32',     # 지역구
    'ta_ymd': 'int32',          # 년월일
    'hour': 'int8',             # 시간대 
    'day': 'int8',              # 요일
    'age': 'int8',              # 연령 
    'sex': 'category',          # 성별
    'amt': 'int32',             # (전수화 보정이 적용된) 매출금액
    'cnt': 'int16'              # (전수화 보정이 적용된) 매출건수
}

# 에러 해결 완료.
def stratified_sampling(df, target_ratio=0.05, random_state=42):

    stratify = ['card_tpbuz_cd', 'admi_cty_no', 'ta_ymd']

    multi = df.duplicated(subset=stratify, keep=False)
    df_multi_groups = df[multi]
    df_single_groups = df[~multi]

    df_sampled = []

    if len(df_multi_groups) > 0 :
        indices = np.arange(len(df))
        try:
            sampled_idx, _ = train_test_split(
                indices, test_size=1 - target_ratio, random_state=random_state,
                stratify=df_multi_groups[stratify].values
            )
            df_sampled.append(df_multi_groups.iloc[sampled_idx])
        except:
            df_sampled.append(df_multi_groups.sample(frac=target_ratio, random_state=random_state))

    if len(df_single_groups) > 0:
        dsg_sample = df_single_groups.sample(frac=target_ratio, random_state=random_state)
        df_sampled.append(dsg_sample)

    if df_sampled:
        return pd.concat(df_sampled, ignore_index=True)
    else:
        return pd.DataFrame(columns=df.columns)


In [11]:
# 모든 월 일괄 샘플링

processed_files = sorted([f for f in os.listdir(DATA_OUTPUT_DIR) if f.startswith('gyeonggi_') and f.endswith('.csv')])

processed_count = 0
error_count = 0

for i, filename in enumerate(processed_files, 1):
    try:
        file_path = os.path.join(DATA_OUTPUT_DIR, filename)
        month = filename.replace('gyeonggi_', '').replace('.csv', '')
        
        print(f"[{i:2d}/{len(processed_files)}] {month}...", end=" ")

        # 파일 읽기
        df = pd.read_csv(file_path, dtype=dtypes)
        
        # 샘플링
        df_sampled = stratified_sampling(df, target_ratio=0.05)
        
        # 저장
        output_file = os.path.join(SAMPLING_OUTPUT_DIR, f'sampled_{month}.csv')
        df_sampled.to_csv(output_file, index=False, encoding='utf-8-sig')
        
        print(f"[{i:2d}/45] {month}: 완료, {len(df_sampled):,}행")
        
        processed_count += 1

        del df, df_sampled
        gc.collect()
    
    except Exception as e:
        print(f"X 오류: {str(e)[:40]}")
        error_count += 1

print(f"샘플링 완료: {processed_count}개월 처리, {error_count}개월 오류")


[ 1/45] 202201... [ 1/45] 202201: 완료, 573,988행
[ 2/45] 202202... [ 2/45] 202202: 완료, 509,563행
[ 3/45] 202203... [ 3/45] 202203: 완료, 605,343행
[ 4/45] 202204... [ 4/45] 202204: 완료, 625,022행
[ 5/45] 202205... [ 5/45] 202205: 완료, 660,912행
[ 6/45] 202206... [ 6/45] 202206: 완료, 637,339행
[ 7/45] 202207... [ 7/45] 202207: 완료, 648,788행
[ 8/45] 202208... [ 8/45] 202208: 완료, 652,387행
[ 9/45] 202209... [ 9/45] 202209: 완료, 627,989행
[10/45] 202210... [10/45] 202210: 완료, 659,279행
[11/45] 202211... [11/45] 202211: 완료, 633,005행
[12/45] 202212... [12/45] 202212: 완료, 645,139행
[13/45] 202301... [13/45] 202301: 완료, 615,025행
[14/45] 202302... [14/45] 202302: 완료, 588,557행
[15/45] 202303... [15/45] 202303: 완료, 670,194행
[16/45] 202304... [16/45] 202304: 완료, 655,180행
[17/45] 202305... [17/45] 202305: 완료, 686,004행
[18/45] 202306... [18/45] 202306: 완료, 670,890행
[19/45] 202307... [19/45] 202307: 완료, 678,276행
[20/45] 202308... [20/45] 202308: 완료, 682,486행
[21/45] 202309... [21/45] 202309: 완료, 655,327행
[22/45] 20231

In [3]:
# 샘플 파일을 연도 별로 병합

# 1. 파일 목록
sampled_files = sorted([f for f in os.listdir(SAMPLING_OUTPUT_DIR) if f.startswith('sampled_') and f.endswith('.csv')])

print("샘플 파일 연도별 병합")
print(f"병합할 파일: {len(sampled_files)}개")

files_by_year = {}

for filename in sampled_files:
    month_str = filename.replace('sampled_', '').replace('.csv', '')
    years = month_str[:4]
    files_by_year.setdefault(years, []).append(filename)

for year in sorted(files_by_year.keys()):
    print(f"  {year}년: {len(files_by_year[year])}개월")

# 2. 병합
for year in sorted(files_by_year.keys()):
    print(f"\n【{year}년 병합】")

    merged_dfs = []

    for filename in sorted(files_by_year[year]):       
        file_path = os.path.join(SAMPLING_OUTPUT_DIR, filename)
        
        month_str = filename.replace('sampled_', '').replace('.csv', '')
        
        print(f"    병합 대상 : {month_str}...", end=" ")

        df = pd.read_csv(file_path)

        merged_dfs.append(df)

        print(f"    완료 :  {len(df):,}행")
        
        gc.collect()

    df_year = pd.concat(merged_dfs, ignore_index=True)

    output_file = os.path.join(SAMPLING_OUTPUT_DIR, f'sampled_{year}.csv')

    df_year.to_csv(output_file, index=False, encoding='utf-8-sig')

    print(f"완료! {len(df_year):,}행")

    print("【검증】")
    print(f"  행: {len(df_year):,}")
    print(f"  열: {len(df_year.columns)}")
    print(f"  중복행: {df_year.duplicated().sum()}")
    print(f"  결측치: {df_year.isnull().sum().sum()}")
    print(f"  파일 크기: {df_year.memory_usage(deep=True).sum() / 1024**3:.2f}GB")
    print(f"\n파일 저장: {output_file}")

    del df_year, merged_dfs
    gc.collect()

print("【병합 완료】")


샘플 파일 연도별 병합
병합할 파일: 45개
  2022년: 12개월
  2023년: 12개월
  2024년: 12개월
  2025년: 9개월

【2022년 병합】
    병합 대상 : 202201...     완료 :  573,988행
    병합 대상 : 202202...     완료 :  509,563행
    병합 대상 : 202203...     완료 :  605,343행
    병합 대상 : 202204...     완료 :  625,022행
    병합 대상 : 202205...     완료 :  660,912행
    병합 대상 : 202206...     완료 :  637,339행
    병합 대상 : 202207...     완료 :  648,788행
    병합 대상 : 202208...     완료 :  652,387행
    병합 대상 : 202209...     완료 :  627,989행
    병합 대상 : 202210...     완료 :  659,279행
    병합 대상 : 202211...     완료 :  633,005행
    병합 대상 : 202212...     완료 :  645,139행
완료! 7,478,754행
【검증】
  행: 7,478,754
  열: 12
  중복행: 0
  결측치: 0
  파일 크기: 2.39GB

파일 저장: ./data/external/sampled_2022.csv

【2023년 병합】
    병합 대상 : 202301...     완료 :  615,025행
    병합 대상 : 202302...     완료 :  588,557행
    병합 대상 : 202303...     완료 :  670,194행
    병합 대상 : 202304...     완료 :  655,180행
    병합 대상 : 202305...     완료 :  686,004행
    병합 대상 : 202306...     완료 :  670,890행
    병합 대상 : 202307...     완료 :  678,276행